# Create Custom Table

This guide demonstrates how to create a custom 3LC Table containing different data types.

The table will contain dummy data for demonstration purposes, and contain the following columns:

- `id` (int): A non-writable unique identifier for each row.
- `name` (string): A name for each row.
- `image` (image): An image for each row.
- `label` (enum-int): A tag for each row.
- `age` (int): An age for each row.
- `weight` (float): A sample-weight for each row.

We show how column schemas can be defined and how to write the Table row-by-row.

In [1]:
import tlc
from tools.common import data_root

In [2]:
# Prepare the data (5 images of dogs with random metadata)

dogs_folder = data_root() / "cats-and-dogs/dogs"

images = [str(dogs_folder / f"150{i}.jpg") for i in range(5)]
names = ["Jennifer", "John", "Jane", "Johnson", "Jenny"]
labels = [0, 1, 1, 0, 0]
ages = [7, 5, 6, 7, 8]
weights = [1, 1, 1, 1, 1]

In [3]:
# Specify the schemas of the columns. 
# The "name" and "age" columns are automatically inferred from the data and do not need to be overridden.

schemas = {
    "id": tlc.Schema(value=tlc.Int32Value(), writable=False),  # Ensure the ID is not writable
    "image": tlc.ImagePath,  # A ImagePath stores paths to images. In the 3LC Dashboard, the images will be displayed.
    "label": tlc.CategoricalLabel("label", classes=["good dog", "bad dog"]),  # Label is just an integer, but we want to display it as a string
    "weight": tlc.SampleWeightSchema(),  # The weight of the sample, to be used for weighted training
}

In [4]:
# Loop over the data and use a tlc.TableWriter to write the table

table_writer = tlc.TableWriter(
    table_name="good-dog-bad-dog",
    dataset_name="Dogs", 
    project_name="3LC Tutorials",
    description="A table of good and bad dogs",
    column_schemas=schemas,
)

for i, (image, name, label, age, weight) in enumerate(zip(images, names, labels, ages, weights)):
    table_writer.add_row(
        {
            "id": i,
            "name": name,
            "image": image,
            "label": label,
            "age": age,
            "weight": weight,
        }
    )

In [5]:
# Finalize the TableWriter to write the table to disk. 
# The URL of the written Table is based on the table name, dataset name, and project name.

table = table_writer.finalize()